In [4]:
import requests
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os.path
from sklearn.svm import SVC
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [5]:
def createFile(response, itemsToRecord, fileName, manipulateData, requiredKeys):
                
    if (os.path.isfile(fileName)):
        os.remove(fileName)    

    with open(fileName,'w') as file:
        file.write('id,')
        for value in itemsToRecord:
            file.write(value+',')
        file.write('\n')

    for stock, periods in response.items():
        dataSetup = []
        for period, periodResult in periods.items():
            temp = {"id":stock+period}
            for item in periodResult:
                if (item) in itemsToRecord:
                    temp[item]=periodResult[item]
            accepted = True
            for requirement in requiredKeys:
                if not requirement in temp.keys():
                    accepted = False
            if accepted:
                dataSetup.append(temp)

        if len(dataSetup) > 0:
            df = pd.DataFrame(dataSetup).set_index("id").sort_index()
            result = manipulateData(df)

        result.to_csv(fileName, mode = 'a', header=False)

In [7]:
url = "http://localhost:7500/edgarQtr"
responsefund = (requests.get(url).json())

ConnectionError: HTTPConnectionPool(host='localhost', port=7500): Max retries exceeded with url: /edgarQtr (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x10c6af748>: Failed to establish a new connection: [Errno 61] Connection refused',))

In [6]:
itemsToRecord = [
"cashandcashequivalents", 
"cashfromfinancingactivities", 
"cashfrominvestingactivities",
"cashfromoperatingactivities",
"costofrevenue",
"ebit",
"grossprofit",
"incomebeforetaxes",
"interestexpense",
"netincome",
"propertyplantequipmentnet",
"sellinggeneraladministrativeexpenses",
"totalassets",
"totalcurrentassets",
"totalliabilities",
"totalcurrentliabilities",
"totallongtermdebt",
"totalreceivablesnet",
"totalrevenue",
"totalshorttermdebt",
"totalstockholdersequity",
] 

requiredFund = []

fileNameFund = "fundamentalReturns.csv"



def manipulateFunction(data):
    return round((data[1:]/data[:-1].values) -1,3)
     
createFile(responsefund, itemsToRecord, fileNameFund, manipulateFunction, requiredFund)

print("return results saved to", fileNameFund)


return results saved to fundamentalReturns.csv


In [15]:
url = "http://localhost:7500/returnsP"
responsePrice = (requests.get(url).json())

In [1]:
itemsToSavePrice = ["0dayPrice", "7dayPrice", "14dayPrice", "30dayPrice", "90dayPrice"]
# itemsToSavePrice = ["0dayPrice", captureValue]

fileNamePrice = "assets/priceReturns.csv"
requiredPrice = ["0dayPrice"]

def manipulateFunction(data):
    results =  round(data.div(data['0dayPrice'], axis='index') - 1,3)
    return results
     
createFile(responsePrice, itemsToSavePrice, fileNamePrice, manipulateFunction, requiredPrice)

print("return results saved to", fileNamePrice)
    


NameError: name 'createFile' is not defined

In [41]:
captureValue = "90dayPrice"
resultThreshhold = 0.03

priceDf = pd.read_csv(fileNamePrice).set_index("id")
fundDf = pd.read_csv(fileNameFund).set_index("id")
priceDf = priceDf[[captureValue]]
joinDf = priceDf.join(fundDf)

# to drop -> joinDf = joinDf.drop(columnName, axis=1)

cols = [c for c in joinDf.columns if c.lower()[:7] != 'unnamed']
joinDf = joinDf[cols]

# output logic    
joinDf[joinDf>5] = 5
joinDf[joinDf<-5] = -5
joinDf.ix[joinDf[captureValue] > resultThreshhold, captureValue] = 1
joinDf.ix[joinDf[captureValue] <= resultThreshhold, captureValue] = 0
joinDf = joinDf.dropna()

print(joinDf.shape)
print(joinDf[captureValue][joinDf[captureValue]==1].count(), 'out of', joinDf[captureValue].count(), 'are positive')



(8665, 22)
4699 out of 8665 are positive


In [3]:
def modelExec(df):
    df = shuffle(df)
    train, test = train_test_split(df, test_size = 0.2)
    x=train.as_matrix(columns=train.columns[1:])
    y=train.as_matrix(columns=train.columns[0:1]).ravel()
    x_=test.as_matrix(columns=test.columns[1:])
    y_=test.as_matrix(columns=test.columns[0:1]).ravel()
    clf = SVC()
    clf.fit(x, y)
    y_pred = clf.predict(x_)
#     print(y_pred.shape)
    return {"cf": confusion_matrix(y_, y_pred), "ac": accuracy_score(y_, y_pred) }

sumAc = 0;
sumCf = np.zeros(shape=(2,2))
numberofIterations = 1
for i in range(numberofIterations):
    executedModel =  modelExec(joinDf)
    sumAc += executedModel["ac"]
    sumCf += executedModel["cf"]

print('TN   FP')
print('FN   TP')
print(sumCf/numberofIterations)
print('accurracy', sumAc/numberofIterations)



NameError: name 'joinDf' is not defined

In [50]:
# output to file
outputFilename = "output.csv"
joinDf.to_csv(outputFilename)
print("created output file")


created output file
